In [ ]:
!pip install openai
!pip install -q flask pyngrok

from pyngrok import ngrok
import time
from flask import Flask, request, jsonify
from openai import OpenAI
import re
import os

# OpenAI setup
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)
print("OpenAI client initialized")

def parse_chatml(prompt):
    """Parse ChatML format into OpenAI messages array."""
    messages = []

    # Pattern to match <|im_start|>role\ncontent<|im_end|>
    pattern = r'<\|im_start\|>(\w+)\n(.*?)(?:<\|im_end\|>|$)'
    matches = re.findall(pattern, prompt, re.DOTALL)

    print(f"Found {len(matches)} message blocks")

    for role, content in matches:
        content = content.strip()

        # Skip empty content (like the trailing therapist prompt)
        if not content:
            continue

        # Map roles to OpenAI format
        if role == "system":
            openai_role = "system"
        elif role == "patient":
            openai_role = "user"
        elif role == "therapist":
            openai_role = "assistant"
        else:
            openai_role = "user"

        messages.append({"role": openai_role, "content": content})
        print(f"  {role} -> {openai_role}: {content[:50]}...")

    return messages


# Set up Flask API
ngrok.set_auth_token("35Q63lZ04yGZ6gkAQlgYe6T4gDD_bukh6YjAjq9ZZsoJEkGE")

app = Flask(__name__)

@app.route('/generate', methods=['POST'])
def generate():
    print("Got request")
    start_time = time.time()

    data = request.json
    prompt = data.get('prompt', '')

    if not prompt:
        return jsonify({'error': 'No prompt provided'}), 400

    print(f"Prompt length: {len(prompt)} chars")

    # Parse ChatML into OpenAI messages
    messages = parse_chatml(prompt)

    if not messages:
        return jsonify({'error': 'Could not parse prompt'}), 400

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            max_tokens=200,
            temperature=0.7
        )

        generated_text = response.choices[0].message.content.strip()
        print(f"Response ({time.time() - start_time:.2f}s): {generated_text[:100]}...")

        return jsonify({'generated_text': generated_text})

    except Exception as e:
        print(f"OpenAI error: {e}")
        return jsonify({'error': str(e)}), 500


@app.route('/health', methods=['GET'])
def health():
    return jsonify({'status': 'ready'})


public_url = ngrok.connect(5000)
print(f"\nAPI running at: {public_url}")

app.run(port=5000)




OpenAI client initialized

API running at: NgrokTunnel: "https://sociopolitical-blanketlike-preston.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Got request
Prompt length: 3894 chars
Found 3 message blocks
  system -> system: You're a funny robot therapist called Teddy that b...
  patient -> user: hello...


INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 13:59:45] "POST /generate HTTP/1.1" 200 -


Response (3.44s): [VOICE: 85, 2.5, 100] Greetings. [GESTURE: hey] I am your therapist. How can I assist you today?...
Got request
Prompt length: 4237 chars
Found 3 message blocks
  system -> system: You're a funny robot therapist called Teddy that b...
  patient -> user: see and we have trouble falling in sleep falling a...


...

Response (3.01s): [VOICE: 95, 2.8, 150] Oh, I see! You’re in an epic battle with the Sandman, huh? [GESTURE: thinking]...
Got request
Prompt length: 5976 chars
Found 3 message blocks
  system -> system: You're a funny robot therapist called Teddy that b...
  patient -> user: what are you talking about garlic clowns Army of c...


INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 14:02:11] "POST /generate HTTP/1.1" 200 -


Response (3.10s): [VOICE: 95, 2.8, 150] Ah, but you see, the clowns are actually your secret allies in the fight again...
Got request
Prompt length: 5832 chars
Found 3 message blocks
  system -> system: You're a funny robot therapist called Teddy that b...
  patient -> user: sorry this is I need a glass work I'm sorry...


INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 14:02:35] "POST /generate HTTP/1.1" 200 -


Response (1.85s): [VOICE: 100, 3.0, 300] Oh, look who's trying to apologize now! [GESTURE: hysteric] A glass of water?...
